In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split


In [19]:
transform = transforms.Compose([transforms.ToTensor()])
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
print("dataset size:", len(dataset))

dataset size: 60000


In [20]:

# Calculate the sizes of each dataset
total_size = len(dataset)
train_size = int(0.6 * total_size)
validate_size = int(0.2 * total_size)
test_size = total_size - train_size - validate_size

# Split the dataset
train_dataset, validate_dataset, test_dataset = random_split(dataset, [train_size, validate_size, test_size])
print("train dataset size :",len(train_dataset))
print("validate dataset size :",len(validate_dataset))
print("test dataset size :",len(test_dataset))

train dataset size : 36000
validate dataset size : 12000
test dataset size : 12000


In [28]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
validateloader = DataLoader(validate_dataset, batch_size=64, shuffle=False)


In [29]:
# Define the Neural Network Model
import torch.nn as nn
import torch.nn.functional as F

class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()
        # Define layers 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)  

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = MobileNet()



In [30]:
# Define the crossEntropy and the Optimizer
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [33]:
# training the model 
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs = 10  


for epoch in range(num_epochs):
    
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data[0].to(device), data[1].to(device)

        
        optimizer.zero_grad()

       
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(trainloader)
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

   


Epoch 1, Train Loss: 0.0083, Train Accuracy: 99.71%
Epoch 2, Train Loss: 0.0062, Train Accuracy: 99.80%
Epoch 3, Train Loss: 0.0072, Train Accuracy: 99.77%
Epoch 4, Train Loss: 0.0094, Train Accuracy: 99.71%
Epoch 5, Train Loss: 0.0069, Train Accuracy: 99.76%
Epoch 6, Train Loss: 0.0061, Train Accuracy: 99.78%
Epoch 7, Train Loss: 0.0060, Train Accuracy: 99.80%
Epoch 8, Train Loss: 0.0027, Train Accuracy: 99.91%
Epoch 9, Train Loss: 0.0070, Train Accuracy: 99.76%
Epoch 10, Train Loss: 0.0044, Train Accuracy: 99.85%


In [35]:
# Validation 
model.eval()  
running_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for data in validateloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss = running_loss / len(validateloader)
val_accuracy = 100 * correct / total
print(f"Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

print('Finished Training')


Epoch 10, Validation Loss: 0.0678, Validation Accuracy: 98.69%
Finished Training
